In [1]:
from pathlib import Path

import pandas as pd

from extract_features_rf import get_combined_features


DATA_FOLDER_PATH = '/Users/debor/repos/PoTeC-data/eyetracking_data/scanpaths_reader_rm_wf'

paths = list(Path(DATA_FOLDER_PATH).glob('*.txt'))

dfs = []

for path in paths:
    df = pd.read_csv(path, sep='\t', na_values=['None', '.'])

    dfs.append(df)

data_arr, feature_names = get_combined_features(dfs)

Extracting numerical features:   0%|          | 0/900 [00:00<?, ?it/s]/Users/debor/repos/PoTeC/extract_features_rf.py:935: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return skew(not_nan_values)
/Users/debor/repos/PoTeC/extract_features_rf.py:938: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return kurtosis(not_nan_values)
Extracting lexical features: 100%|██████████| 900/900 [2:04:19<00:00,  8.29s/it]    


In [2]:
pd.DataFrame(data_arr, columns=feature_names)

,CURRENT_FIX_DURATION_mean,CURRENT_FIX_DURATION_std,CURRENT_FIX_DURATION_median,CURRENT_FIX_DURATION_skew,CURRENT_FIX_DURATION_kurtosis,roi_mean,roi_std,roi_median,roi_skew,roi_kurtosis,...,lexical_word_embedding_spacy_lemma_86,lexical_word_embedding_spacy_lemma_87,lexical_word_embedding_spacy_lemma_88,lexical_word_embedding_spacy_lemma_89,lexical_word_embedding_spacy_lemma_90,lexical_word_embedding_spacy_lemma_91,lexical_word_embedding_spacy_lemma_92,lexical_word_embedding_spacy_lemma_93,lexical_word_embedding_spacy_lemma_94,lexical_word_embedding_spacy_lemma_95
0,196.361343,92.259651,187.0,1.268841,5.181818,545.978149,313.589050,561.0,-0.015030,-1.201969,...,0.348818,-0.098345,-0.415426,-0.136097,-0.473930,-0.365978,-0.499586,0.279461,0.871738,0.217742
1,196.340973,67.509064,190.0,2.542669,15.811152,460.945557,281.162292,442.0,0.454465,-0.653694,...,0.311055,-0.096699,-0.408950,-0.165718,-0.402917,-0.339687,-0.465964,0.272932,0.813951,0.174176
2,209.595139,80.511436,194.5,1.836004,8.130287,410.882599,264.934082,408.0,0.177709,-1.173818,...,0.303842,-0.097198,-0.291308,-0.141470,-0.574067,-0.313650,-0.507244,0.360266,0.838996,0.177530
3,171.326538,87.716637,159.0,2.967913,14.168442,561.787170,278.160645,559.0,-0.071132,-0.970753,...,0.111021,-0.017154,-0.451527,-0.222214,-0.303739,-0.304234,-0.564872,0.246955,0.818862,0.318269
4,213.942856,65.469948,214.0,0.668290,2.336957,449.279358,264.568909,455.0,0.025539,-1.157084,...,0.291815,-0.069483,-0.323504,-0.185709,-0.563363,-0.291159,-0.484449,0.320729,0.841880,0.214453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,220.777771,81.548592,209.5,2.567144,14.898708,535.138916,276.385071,556.5,-0.265825,-1.135572,...,0.069051,-0.093141,-0.414736,-0.077777,-0.396381,-0.383954,-0.466554,0.294311,1.019377,0.194179
896,239.742432,121.603416,210.5,2.166776,7.322240,491.271057,305.496948,482.0,0.059148,-1.184529,...,0.237014,-0.054264,-0.336871,-0.231749,-0.613936,-0.339112,-0.522419,0.259881,0.984458,0.147186
897,198.405029,124.120804,178.5,5.289953,49.230427,461.932953,341.265015,348.0,0.451358,-1.100906,...,0.309779,-0.069427,-0.314597,-0.191150,-0.363702,-0.282544,-0.431772,0.223544,0.887344,0.330846
898,196.111115,70.513847,184.0,1.269712,3.416124,489.241821,276.006836,468.0,0.171846,-0.985881,...,0.317854,-0.122346,-0.464454,-0.088481,-0.569035,-0.303349,-0.467401,0.238378,0.991982,0.158073


In [27]:
len(feature_names)

391

In [4]:
# get the reader id, the group, item id and the expert level from all the dfs in the list
reader_ids = [df['reader'].iloc[0] for df in dfs]
groups = [df['group'].iloc[0] for df in dfs]
expert_levels = [df['expert_status'].iloc[0] for df in dfs]
item_ids = [df['itemid'].iloc[0] for df in dfs]
print(len(reader_ids), len(groups), len(expert_levels), len(item_ids))

900 900 900 900


In [5]:
reader_ids[890:]

[95, 22, 1, 75, 73, 80, 7, 75, 1, 79]

In [6]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

from sklearn.ensemble import RandomForestClassifier

RANDOM_STATE = 21

np.random.seed(RANDOM_STATE)
pd.set_option('display.max_columns', None)

In [7]:
X = data_arr
y = np.array(expert_levels)

In [8]:
if len(np.unique(y)) == 4:
    print(f'Num bio-experts: {np.count_nonzero(y == 2)}')
    print(f'Num bio-beginners: {np.count_nonzero(y == 1)}')
    print(f'Num physik-experts: {np.count_nonzero(y == 4)}')
    print(f'Num physik-beginners: {np.count_nonzero(y == 3)}')

else:
    experts = np.count_nonzero(y == 1)
    beginners = np.count_nonzero(y == -1)
    print(f'Num experts: {experts}')
    print(f'Num beginners: {beginners}')
    print(f'Accuracy majority voting: {(experts if experts > beginners else beginners) / (experts + beginners)}')

Num experts: 564
Num beginners: 336
Majority voting: 0.6266666666666667


In [9]:
param_grid = {
    'n_estimators': [50, 100, 300, 400, 500, 700, 900, 1000, 1200],
    'max_features': ['sqrt', 'log2', None],
    'max_depth' : [2, 4, 8, 16, 32, 64, None],
    'criterion' :['entropy', 'gini', 'log_loss'],
    'random_state': [RANDOM_STATE],
    'n_jobs': [-1]
}
grid_search_verbosity = 1

In [11]:
import logging


class DebugFilter(logging.Filter):
    def filter(self, record):
        return record.levelno == logging.DEBUG

def get_logger():
    
        # configuration for log file
    log = logging.getLogger('rf-baseline-grid-search')
    logger.setLevel(logging.DEBUG)
    
    # configure separate logging handlers for info and debug to separate files
    info_log = logging.FileHandler(f'hparams_search/grid_search_rf_baseline_info_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log')
    info_log.setLevel(logging.INFO)
    info_log.setFormatter(logging.Formatter('%(asctime)s %(levelname)s %(message)s', datefmt='%Y-%m-%d %H:%M:%S'))
    
    debug_log = logging.FileHandler(f'hparams_search/grid_search_rf_baseline_debug_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log')
    debug_log.setLevel(logging.DEBUG)
    debug_log.setFormatter(logging.Formatter('%(asctime)s %(levelname)s %(message)s', datefmt='%Y-%m-%d %H:%M:%S'))
    # configure filter to ignore all messages higher than debug for the debug file
    debug_log.addFilter(DebugFilter())
    
    logging.getLogger().addHandler(info_log)
    logging.getLogger().addHandler(debug_log)
        
    # log info logger filename to debug handler
    log.debug(f'Info logger filename: {info_log.baseFilename}')
        
    
    return log

In [26]:
from datetime import datetime
from sklearn.model_selection import GridSearchCV

logger = get_logger()

kf = StratifiedGroupKFold(n_splits=5)

split_criterion = reader_ids
split_criterion_str = 'reader'
scores = []
outer_fold = 1
best_score = 0
best_hparams = None

logger.info(f'Grid search for random forest baseline. New {split_criterion_str} split.\nStarted at {datetime.now()}')
logger.info(f'Grid search parameters: {param_grid}')


for train_index, test_index in kf.split(X, y, groups=split_criterion):
    logger.info(f'Outer fold {outer_fold}')
    logger.debug(f'Outer fold {outer_fold}')
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    outer_train_ids = [split_criterion[i] for i in train_index]
    outer_test_ids = [split_criterion[i] for i in test_index]
    
    logger.info(f'Outer train {split_criterion_str}-ids: {sorted(set(outer_train_ids))}')
    logger.info(f'Outer test {split_criterion_str}-ids: {sorted(set(outer_test_ids))}')
    logger.debug(f'Outer train data indices: {train_index}')
    logger.debug(f'Outer test data indices: {test_index}')
    logger.debug(f'Outer train data labels: {y_train}')
    logger.debug(f'Outer test data labels: {y_test}')    
    logger.debug(f'Inner train {split_criterion_str}-ids: {outer_train_ids}')
    logger.debug(f'Inner test {split_criterion_str}-ids: {outer_train_ids}')
    
    inner_kf = StratifiedGroupKFold(n_splits=5)

    # standardize/normalize the data
    scaler = StandardScaler()
    # fit data on training set
    X_train_std = scaler.fit_transform(X_train)
    # standardize test data as well
    X_test_std = scaler.transform(X_test)
    
    cv_list = list(inner_kf.split(X_train_std, y_train, groups=outer_train_ids))
    
    for inner_fold_idx, (train, test) in enumerate(cv_list):
        inner_train_ids = [outer_train_ids[i] for i in train]
        inner_test_ids = [outer_train_ids[i] for i in test]
        
        logger.info(f'Inner fold {inner_fold_idx + 1}')
        logger.debug(f'Inner fold {inner_fold_idx + 1}')
        
        logger.info(f'Inner train {split_criterion_str}-ids: {sorted(set(inner_train_ids))}')
        logger.info(f'Inner test {split_criterion_str}-ids: {sorted(set(inner_test_ids))}')
        
        logger.debug(f'Inner train data indices on outer train data: {train}')
        logger.debug(f'Inner test data indices on outer train data: {test}')
        logger.debug(f'Inner train data labels on outer train data: {y_train[train]}')
        logger.debug(f'Inner test data labels on outer train data: {y_train[test]}')
        logger.debug(f'Inner train {split_criterion_str}-ids: {inner_train_ids}')
        logger.debug(f'Inner test {split_criterion_str}-ids: {inner_test_ids}')
        

    rf_clf = GridSearchCV(
        estimator=RandomForestClassifier(),
        param_grid=param_grid,
        verbose=grid_search_verbosity,
        scoring='roc_auc',
        return_train_score=True,
        cv=cv_list,
    )

    rf_clf.fit(X_train_std, y_train)
    
    # get best score from estimator on that fold and compare to old best score
    current_best_score = rf_clf.best_score_
    logger.info(f'Best score on inner fold: {current_best_score}')
    #logger.info(f"CV results on outer fold {outer_fold} for best model {rf_clf.cv_results_['params'][rf_clf.best_index_]}")
    logger.debug(f'All CV results on fold {outer_fold}: {rf_clf.cv_results_}')
    
    if current_best_score > best_score:
        logger.info(f'New best score!')
        
        best_score = current_best_score
        best_hparams = rf_clf.best_params_
        logger.info(f'New best parameters on fold {outer_fold}: {rf_clf.best_params_}')
    else:
        logger.info(f'Best parameters still the same on fold {outer_fold}: {best_hparams}')
        
    best_rf = RandomForestClassifier(**best_hparams)
    best_rf.fit(X_train_std, y_train)
    predictions = best_rf.predict(X_test_std)
    score = roc_auc_score(y_test, predictions)
    scores.append(score)
    logger.info(f'Test AUC-ROC on fold {outer_fold} with best hparams: {score}')
    outer_fold += 1

final_mean_score = round(np.array(scores).mean(), 4)
print(f'Final mean score: {final_mean_score}')
logger.info(f'Final AUC-ROC score (mean over 5 folds): {final_mean_score}')

Fitting 5 folds for each of 567 candidates, totalling 2835 fits
Fitting 5 folds for each of 567 candidates, totalling 2835 fits


/Users/debor/opt/anaconda3/envs/PoTeC/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/debor/opt/anaconda3/envs/PoTeC/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Fitting 5 folds for each of 567 candidates, totalling 2835 fits
Fitting 5 folds for each of 567 candidates, totalling 2835 fits


/Users/debor/opt/anaconda3/envs/PoTeC/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/debor/opt/anaconda3/envs/PoTeC/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/debor/opt/anaconda3/envs/PoTeC/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/Users/debor/opt/anaconda3/envs/PoTeC/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to t

Fitting 5 folds for each of 567 candidates, totalling 2835 fits


/Users/debor/opt/anaconda3/envs/PoTeC/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Final mean score: 50.94


In [31]:
final_kf = StratifiedGroupKFold(n_splits=5)

final_scores = []
for train_index, test_index in final_kf.split(X, y, groups=split_criterion):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # standardize/normalize the data
    scaler = StandardScaler()
    # fit data on training set
    X_train_std = scaler.fit_transform(X_train)
    # standardize test data as well
    X_test_std = scaler.transform(X_test)
    
    
    rf = RandomForestClassifier(**{'criterion': 'gini', 'max_depth': 32, 'max_features': None, 'n_estimators': 700, 'n_jobs': -1, 'random_state': 21})
    rf.fit(X_train_std, y_train)
    predictions = rf.predict(X_test_std)
    score = roc_auc_score(y_test, predictions)
    final_scores.append(score)

round(np.array(final_scores).mean(), 4)

0.5104

In [ ]:
metric_dict = {'dummy':{'auc':[],
                        'fprs':[],
                        'tprs':[],
                        'proba':[],
                        'label':[]},
                'rf':{  'auc':[],
                        'fprs':[],
                        'tprs':[],
                        'proba':[],
                        'label':[]},
              }
for f in range(folds):
    # TODO: redo as stratifiedkfold over subjects
    subjkeys = list(set(label_arr[:, 0]))
    # set seeds (maybe define it as utils)
    random.seed(f)
    np.random.seed(f)
    subjkeys = np.random.permutation(subjkeys) #random shuffling
    
    N_totalsub = len(subjkeys)
    N_trainsub = round(0.8*N_totalsub)
    N_validsub = round(0.1*N_totalsub)
    N_testsub = N_totalsub - N_trainsub - N_validsub


    N_train_sub = subjkeys[:N_trainsub]
    N_val_sub = subjkeys[N_trainsub:N_trainsub + N_validsub]
    N_test_sub = subjkeys[N_trainsub + N_validsub: N_trainsub + N_validsub + N_testsub]
    X_train = data_arr_rf[np.isin(label_arr[:, 0], N_train_sub)]
    X_val = data_arr_rf[np.isin(label_arr[:, 0], N_val_sub)]
    X_test = data_arr_rf[np.isin(label_arr[:, 0], N_test_sub)]
    y_train = label_arr[np.isin(label_arr[:, 0], N_train_sub)]
    y_val = label_arr[np.isin(label_arr[:, 0], N_val_sub)]
    y_test = label_arr[np.isin(label_arr[:, 0], N_test_sub)]
        
    # baseline 
    dummy_clf = DummyClassifier(strategy="most_frequent")
    dummy_clf.fit(X_train,np.array(y_train[:, 2], dtype=int))
    dummy_proba = dummy_clf.predict_proba(X_test)
    fpr, tpr, _ = metrics.roc_curve(np.array(y_test[:, 2], dtype=int), dummy_proba[:,1], pos_label=1)
    auc = metrics.auc(fpr, tpr)
    
    metric_dict['dummy']['auc'].append(auc)
    metric_dict['dummy']['fprs'].append(fpr)
    metric_dict['dummy']['tprs'].append(tpr)
    metric_dict['dummy']['proba'].append(dummy_proba)
    metric_dict['dummy']['label'].append(y_test)
    
    # rf
    pred_proba, best_parameters, rf = evaluate_rf.evaluate_rf(X_train, X_val, X_test,
                                                          np.array(y_train[:, 2], dtype=int), 
                                                          np.array(y_val[:, 2], dtype=int))
    fpr, tpr, _ = metrics.roc_curve(np.array(y_test[:, 2], dtype=int), pred_proba[:,1], pos_label=1)
    auc = metrics.auc(fpr, tpr)
    metric_dict['rf']['auc'].append(auc)
    metric_dict['rf']['fprs'].append(fpr)
    metric_dict['rf']['tprs'].append(tpr)
    metric_dict['rf']['proba'].append(pred_proba)
    metric_dict['rf']['label'].append(y_test)
    
    
    #print('auc: ' + str(auc))
    aucs.append(auc)
    fprs.append(fpr)
    tprs.append(tpr)
    
    baseline_accs.append(dummy_clf.score(X_test, np.array(y_test[:, 2], dtype=int)))
    rf_accs.append(rf.score(X_test, np.array(y_test[:, 2], dtype=int)))
    print('baseline: ' + str(baseline_accs[-1]) + ' vs. ' + str(rf_accs[-1]))